In [1]:

import numpy as np

import matplotlib.pyplot as plt

import collections

from functools import reduce

from multiprocess import Pool
from functools import partial
import numpy as np

from itertools import islice
import games
import feedexp3
import geometry_v2
import cpb
import cpb_uniform
import cpb_gaussian
import feedexp3_v2
import bpm
import random_algo
import feedexp3_v3
import plotly.graph_objects as go
import TSPM
import eTSPM
import ucbtspm

def evaluate_parallel(nbCores, n_folds, horizon, alg, game, type):
    print("nbCores:", nbCores, "nbFolds:", n_folds, "Horizon:", horizon)
    pool = Pool(processes = nbCores) 
    task = Evaluation(horizon)

    np.random.seed(2)
    distributions = []

    for jobid in range(n_folds):
        p = get_easy() if type == 'easy' else get_harsch() 
        distributions.append( p )

    return np.asarray(  pool.map( partial( task.eval_policy_once, alg, game ), zip(distributions ,range(n_folds)) ) ) 

def get_easy(c = 2):

    res = False
    while res == False:
        pbt = np.random.uniform(0,1,5)
        pbt = pbt / pbt.sum()
        test = np.array([ pbt[i]>=c/(c+1) * pbt[i-1] for i in range(1, 5)])
        res = test.all()
    return pbt

def get_harsch(c = 2):

    res = True
    while res:
        pbt = np.random.uniform(0,1,5)
        pbt = pbt / pbt.sum()
        test = np.array([ pbt[i]>=c/(c+1) * pbt[i-1] for i in range(1, 5)])
        if (test == False).any():
            res = False
    return pbt

class Evaluation:

    def __init__(self, horizon ):
        self.horizon = horizon
        # self.outcome_distribution = outcome_distribution

    def get_outcomes(self, game, job_id):
        # self.means = runif_in_simplex( self.game.n_outcomes )
        outcomes = np.random.choice( game.n_outcomes , p= list( game.outcome_dist.values() ), size= self.horizon) 
        return outcomes

    def get_feedback(self, game, action, outcome):
        return game.FeedbackMatrix[ action ][ outcome ]

    def eval_policy_once(self, alg, game, job):

        p, jobid = job
        #print(p)

        np.random.seed(jobid)

        outcome_distribution =  {'a':p[0],'b':p[1],'c':p[2],'d':p[3],'e':p[4]}

        game.set_outcome_distribution( outcome_distribution )
        # print('optimal action', game.i_star)

        action_counter = np.zeros( (game.n_actions, self.horizon) )

        # generate outcomes obliviously
        outcomes = self.get_outcomes(game, jobid)

        for t in range(self.horizon):

            # policy chooses one action
            action = alg.get_action(t)

            # Environment chooses one outcome
            outcome = outcomes[t]

            # print('t', t, 'action', action, 'outcome', outcome, )

            feedback =  self.get_feedback( game, action, outcome )

            alg.update(action, feedback, outcome)
            
            # print('nu', alg.nu / alg.n )

            for i in range(game.n_actions):
                if i == action:
                    action_counter[i][t] = action_counter[i][t-1] +1
                else:
                    
                    action_counter[i][t] = action_counter[i][t-1]

        regret = np.array( [ game.delta(i) for i in range(game.n_actions) ]).T @ action_counter

        return regret


In [3]:

n_cores = 1
n_folds = 1
horizon = 150
type = 'hard'
nfolds = 100

np.random.seed(2)
distributions = []

for jobid in range(nfolds):
    p = get_easy() if type == 'easy' else get_harsch() 
    distributions.append( p )

# np.seterr(all='raise')
for _ in range(nfolds):
    game =  games.dynamic_pricing( 'harsch' ) 
    alg = TSPM.TSPM_alg(game, horizon, 1 )
    task = Evaluation(horizon,)
    print(  distributions[_] )
    result = task.eval_policy_once(alg,game, (    distributions[_], 2) )

#plt.plot(range(horizon), result)
# fig = go.Figure( )
# regret = np.array([ game.delta(i) for i in range(game.n_actions) ]).T @ np.mean(result,0) 
# xcoords = np.arange(0,horizon,1).tolist()

# fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='blue'), mode='lines',  name='TPSM' )) # 

n-actions 5 n-outcomes 5 alphabet 2
[0.23349275 0.01388454 0.2943663  0.2331326  0.22512381]
[0.23349275 0.01388454 0.2943663  0.2331326  0.22512381]
n-actions 5 n-outcomes 5 alphabet 2
[0.19197294 0.11893085 0.35988718 0.17414328 0.15506575]
[0.19197294 0.11893085 0.35988718 0.17414328 0.15506575]
n-actions 5 n-outcomes 5 alphabet 2
[0.31324929 0.26685616 0.06787116 0.25900696 0.09301644]
[0.31324929 0.26685616 0.06787116 0.25900696 0.09301644]
n-actions 5 n-outcomes 5 alphabet 2
[0.25666609 0.27909931 0.16152828 0.2766763  0.02603002]
[0.25666609 0.27909931 0.16152828 0.2766763  0.02603002]
n-actions 5 n-outcomes 5 alphabet 2
[0.41334137 0.05341083 0.35024649 0.07897185 0.10402946]
[0.41334137 0.05341083 0.35024649 0.07897185 0.10402946]
n-actions 5 n-outcomes 5 alphabet 2
[0.39787384 0.15069119 0.07130494 0.14688691 0.2332431 ]
[0.39787384 0.15069119 0.07130494 0.14688691 0.2332431 ]
n-actions 5 n-outcomes 5 alphabet 2
[0.20354126 0.08778147 0.27865045 0.21019084 0.21983598]
[0.2035

In [2]:
n_cores = 8
n_folds = 100
horizon = 5000

import ucbTSPM_v2
import cpb_gaussian
import cpb_uniform
game = games.dynamic_pricing( 'harsch' )

algos = [   TSPM.TSPM_alg(game, horizon , 1),
        TSPM.TSPM_alg(game, horizon, 0 ) ,
        random_algo.Random(  game, horizon, ),      
        cpb.CPB(  game, horizon, 1.01), 
        cpb_uniform.CPB_uniform(  game, horizon, 1.01), 
        cpb_gaussian.CPB_gaussian(  game, horizon, 1.01), ]  #eTSPM.eTSPM_alg(game, horizon, 1)

colors = [     [0,255,0], [0 , 0 , 255], [0,0,0], [255,0,0], [150,0,0],  [150,100,0],  ] # [155,155,0], [255,0,0], [0,0,255] , [255,51,255], [255,51,255], [255,20,200]  ] #
labels = [  'TSPM (R=1)', 'TSPM (R=0)', 'random', 'CPB', 'CPB uniform','CPB gaussian',  ]  # ,'TSPM' , 'ucbTSPM (Auer)' 'FeedExp3 (2001)', 'FeedExp3 (2006)', 'CPB',  'eTSPM (Auer)',

fig = go.Figure( )

for alg, color, label in zip( algos, colors, labels):

    r,g,b = color
    result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'hard' )
    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=True )

    )
    
fig.show(legend=True)

fig.update_yaxes(range=[0, 100] )
fig.show()

n-actions 5 n-outcomes 5 alphabet 2
n-actions 5 n-outcomes 5 alphabet 2
Restricted license - for non-production use only - expires 2023-10-25
nbCores: 8 nbFolds: 100 Horizon: 5000
limit threshold  173
limit threshold  960
limit threshold  1784
limit threshold  2963
limit threshold  3036
limit threshold  3386
limit threshold  3661
limit threshold  3744
limit threshold  4009
limit threshold  4406
limit threshold  4415
limit threshold  4697
limit threshold  4928
limit threshold  74
limit threshold  530
limit threshold  840
limit threshold  920
limit threshold  1084
limit threshold  1804
limit threshold  1995
limit threshold  2257
limit threshold  2279
limit threshold  2580
limit threshold  2589
limit threshold  2650
limit threshold  2708
limit threshold  3152
limit threshold  194
limit threshold  244
limit threshold  414
limit threshold  501
limit threshold  2346
limit threshold  462
limit threshold  472
limit threshold  2366
limit threshold  2374
limit threshold  679
limit threshold  238

Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-8:
Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-7:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/maximeheui

KeyboardInterrupt: 

  File "/Users/maximeheuillet/Desktop/attack-detection/partial_monitoring/TSPM.py", line 69, in accept_reject
    p_tilde = self.sample_from_g()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/Users/maximeheuillet/Desktop/attack-detection/partial_monitoring/TSPM.py", line 61, in sample_from_g
    if self.in_simplex(p):
  File "/Users/maximeheuillet/Desktop/attack-detection/partial_monitoring/TSPM.py", line 36, in in_simplex
    if p.sum()<=1 and (p<=1).all() and (p>=0).all():
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py", line 63, in _all
    return umr_all(a, axis, dtype, out, keepdims)
KeyboardInterrupt


In [3]:
fig.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig.write_image("./hard_DP.pdf")

In [6]:
n_cores = 8
n_folds = 100
horizon = 5000

game = games.dynamic_pricing( 'easy' )

algos = [   TSPM.TSPM_alg(game, horizon , 1),
        TSPM.TSPM_alg(game, horizon, 0 ) ,
        random_algo.Random(  game, horizon, ),      
        cpb.CPB(  game, horizon, 1.01), 
        cpb_uniform.CPB_uniform(  game, horizon, 1.01), 
        cpb_gaussian.CPB_gaussian(  game, horizon, 1.01), ]  #eTSPM.eTSPM_alg(game, horizon, 1)

colors = [  [0,255,0], [0 , 0 , 255], [0,0,0], [255,0,0], [150,0,0],  [150,100,0],  ] # [155,155,0], [255,0,0], [0,0,255] , [255,51,255], [255,51,255], [255,20,200]  ] #
labels = [  'TSPM (R=1)', 'TSPM (R=0)', 'random', 'CPB', 'CPB uniform','CPB gaussian',  ]  # ,'TSPM' , 'ucbTSPM (Auer)' 'FeedExp3 (2001)', 'FeedExp3 (2006)', 'CPB',  'eTSPM (Auer)',


fig = go.Figure( )

fig = go.Figure( )

for alg, color, label in zip( algos, colors, labels):

    r,g,b = color
    result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'easy')
    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=True )  )
    
fig.show(legend=True)
fig.update_yaxes(range=[0, 150] )
fig.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), font=dict(size=13,) )
fig.write_image("./easy_DP.pdf")
fig.show()

n-actions 5 n-outcomes 5 alphabet 2
n-actions 5 n-outcomes 5 alphabet 2
nbCores: 8 nbFolds: 100 Horizon: 5000
limit trial exceeded


Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._targe

KeyboardInterrupt: 

  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/synchronize.py", line 101, in __enter__
    return self._semlock.__enter__()
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/synchronize.py", line 101, in __enter__
    return self._semlock.__enter__()
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/synchronize.py", line 101, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
Traceback (most recent call last):
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/maximeheuillet/opt/anaconda3/lib/python3.9/site-packages/multiprocess/pool.py", line 125, in w